In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![title](https://github.com/c920720a/MyDataAnalysis_2022/blob/main/title.png?raw=true)

### 01. 소개 및 개요
* 이 대회는 기본 예측(default prediction) 대회이다.
* American Express(AMEX) 에서 개최한 대회로 상금과 채용 특전이 있다.

#### 이 대회의 **설명(Description)**은 다음과 같다.
* How do card issuers know we’ll pay back what we charge? (카드사는 우리가 청구한 금액을 반환할 것을 어떻게 알 수 있는가?)
* Credit default prediction is central to managing risk in a consumer lending business. (신용 채무 불이행 예측은 대출사업에서 위험관리의 핵심이다.)
* In this competition, you’ll apply your machine learning skills to predict credit default. (이 대회에서는 머신러닝을 적용해 신용불이행을 예측한다.)
* you will leverage an industrial scale data set (산업 규모의 데이터셋을 적용한다.) Training, validation, and testing datasets include time-series behavioral data and anonymized customer profile information. (시계열 데이터와 익명화된 고객 프로필 정보기 포함되어 있다.) → 데이터셋이 클 것 같다.
* 최종 마감일은 8월 24일

#### **데이터**를 살펴보자.
* 월별 고객 프로필을 기반으로 향후 신용카드 청구액(사용액)을 갚지 않을 확률을 예측하는 것.
* 최근 신용카드 명세서 이후 18개월 실척을 관찰하여 target값이 계산되며 고객이 가장 최근 명세서 날짜 이후 120일 동안 미납 금액을 지불하지 않으면 기본 이벤트로 간주함.
* 일반 범주 ============================================================
    - D_* = 연체 변수
    - S_* = 지출 변수
    - P_* = 지불 변수
    - B_* = 균형 변수
    - R_* = 위험 변수
* 기능이 익명화되고 정규화됨. Features are anonymized and normalized
* categorical ====================================
     - ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
* 각각 customer_ID에 대해 향후 지불 불이행 가능성(target = 1)을 예측하는 것
* train_data와 train_labers, test_data, submission 파일로 이루어짐.
* negative class(target=0)에 대한 언더샘플링도 되어있다.
* **단, 데이터의 크기(50.31 GB)가 크다. train_data.csv(16.39 GB), test_data.csv(33.82 GB)이다.**

### 참고 Discussion :  **How To Reduce Data Size**
* How To Reduce Data Size 페이지 [링크](https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054)
* 1단계 : 데이터 유형 줄이기
    - customer_ID를 각각 행당 64바이트에서 4바이트로 줄인다.
    - S_2 : S_2는 시간이 있는 날짜 열이다. 행당 10바이트를 차지. 이 열을 pd.to_datetime() 로 변환하면 4 바이트가 된다.
    - 11개의 범주형을 88바이트에서 11바이트로 줄인다.
    - 177 숫자열 : 1416바이트를 353바이트로 줄인다.
##### 
* 2단계 : 파일 형식 선택 
    - csv파일은 데이터를 행별로 저장하고 parquet파일은 데이터를 열별로 저장
    - 압축하기. 50GB -> 6GB
#####  
* 3단계 : 데이터를 여러개의 파일로 저장할 것인지 큰 파일 1개로 저장할 것인지
#####  
* amex-data-integer-dtypes-parquet-format를 사용하여 EDA를 진행해본다.
* gc.collect()는 더이상 사용되지 않는 메모리를 해제시켜 메모리를 줄여주는 역할을 하는 메서드이다.

### 02. 라이브러리 불러오기

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings, gc
warnings.filterwarnings("ignore")

### 03. 데이터 불러오기


In [4]:
%%time
train = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")
label = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train = train.merge(label,how='inner',on="customer_ID")

* 해당 데이터는 .csv에서 직접 읽는 것이 메모리 전체를 소모하기 때문에
   * 데이터를 청크로 쪼개 청크 읽기와
   * 압축된 데이터세트 사용한다.

### 03-01. 청크로 읽기

In [5]:
TRAIN_DATA_PATH = "../input/amex-default-prediction/train_data.csv"

In [6]:
chunksize = 13000
df_train_raw_chunks = pd.read_csv(TRAIN_DATA_PATH, chunksize=chunksize)

In [7]:
df_train_raw_ch = df_train_raw_chunks.__next__()

In [8]:
df_train_raw_ch.head()

In [9]:
sample_customer_id = np.random.choice(df_train_raw_ch['customer_ID'])
customer_data_ex = df_train_raw_ch[df_train_raw_ch["customer_ID"] == sample_customer_id]
customer_data_ex.head()

In [10]:
del df_train_raw_chunks, df_train_raw_ch, customer_data_ex, sample_customer_id
gc.collect()

### 03-02. 압축된 데이터셋 읽기
* 03. 데이터 불러오기에서 읽은 것처럼 Amex Competition Data in Parquet Format 불러오기

In [11]:
train_raw = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')

In [12]:
labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [13]:
labels.head()

In [14]:
train_raw = train_raw.merge(labels, left_on='customer_ID', right_on='customer_ID')

In [15]:
train_raw.shape

### 04. EDA(Exploratory Data Analysis : 탐색적 데이터 분석)
* 모델이나 특징을 만들 때 가장 먼저 데이터 이해가 필요함.
* 사전에 가설이나 모델을 상정하지 않은 상태에서 데이터에 관한 이해를 높이고자 다양한 관점에서 데이터를 살펴보는 과정을 탐색적 데이터 분석(EDA)라고 함.
* 어떤 컬럼이 있는지, 각 컬럼의 형태는 어떠한지, 값의 분포나 결측값 / 이상치는 어떻게 되어있는지 이해하며, 목적변수와 각 변수의 상관관계를 보는 것.

* 통계량으로서 EDA
  * 변수의 평균, 표준편차, 최댓값, 최솟값, 분위수
  * 범주형 변수의 종류 수
  * 변수의 결측값 수
  * 변수 간 상관계수

* 사용되는 시각화 기법
  * bar graph, box plot, violin plot, scatter plot, broken line graph, heatmap, histogram, Q-Q plot, t-SNE, UMAP 등
  
 

[출처] 데이터가 뛰어노는 AI 놀이터,캐글

* 일반 범주 ============================================================
    * D_* = 연체 변수 Delinquency variables
    * S_* = 지출 변수 Spend variables
    * P_* = 지불 변수 Payment variables
    * B_* = 균형 변수 Balance variables
    * R_* = 위험 변수 Risk variables
##### 
* 기능 범주 : 익명화되고 정규화됨. ========================================
    * ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']<br>
##### 
* S_2: contains a timestamp 시계열 데이터

In [16]:
train_raw['S_2'] = pd.to_datetime(train_raw['S_2'])

In [17]:
categorical_features = ['B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']
train_raw[categorical_features] = train_raw[categorical_features].astype("category")
train_raw[categorical_features].dtypes

In [18]:
print(f'Train dates range is from {train_raw["S_2"].min()} to {train_raw["S_2"].max()}.')

* 5,531,451개의 열
* 191개의 feature값
* 178개의 cloumns
* 1개의 datetime columns, 2017-03-01 부터 2018-03-31 까지의 데이터

### 04-01. Missing data 결측치 찾기

In [19]:
tmp = train_raw.isna().sum().div(len(train_raw)).mul(100).sort_values(ascending=False)

In [20]:
plt.style.use('Solarize_Light2')
fig, ax = plt.subplots(2,1, figsize=(25,10))
sns.barplot(x=tmp[:100].index, y=tmp[:100].values, ax=ax[0])
sns.barplot(x=tmp[100:].index, y=tmp[100:].values, ax=ax[1])
ax[0].set_ylabel("Percentage [%]"), ax[1].set_ylabel("Percentage [%]")
ax[0].tick_params(axis='x', rotation=90); ax[1].tick_params(axis='x', rotation=90)
plt.suptitle("Amount of missing data")
plt.tight_layout()
plt.show()

### 04-02. target 변수들의 분포 살펴보기

In [21]:
tmp = train_raw['target'].value_counts().div(len(train_raw)).mul(100)

In [22]:
ax = sns.barplot(x=tmp.index, y=tmp.values)
ax.bar_label(ax.containers[0], fmt='%.f%%')
plt.title("Distribution of a target variable")
plt.ylabel("Percentage [%]")
plt.show()

* 향후 채무 불이행 가능성(target=1) 기본 상태는 25%이다.

In [23]:
print(f'Number of unique customers: {train_raw["customer_ID"].nunique()}')

* 고유 고객의 수는 458,913명이다.

In [24]:
cust_presence = train_raw.groupby(['customer_ID','target']).size().reset_index().rename(columns={0:'presence'})

In [25]:
cust_presence.head()

In [26]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
sns.histplot(x='presence', data=cust_presence, hue='target', stat='percent', multiple="dodge", bins=np.arange(0,14), ax=ax)
ax.bar_label(ax.containers[0], fmt='%.f%%')
ax.bar_label(ax.containers[1], fmt='%.f%%')
plt.show()

In [27]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
sns.histplot(x='presence', data=cust_presence, hue='target', stat='percent', multiple="dodge", bins=np.arange(0,14), ax=ax)
ax.bar_label(ax.containers[0], fmt='%.2f%%')
ax.bar_label(ax.containers[1], fmt='%.2f%%')
ax.set_xlim(0,12)
ax.set_ylim(0,1)
plt.show()

* database에 있는 시간이 짧은 고객이 이탈할 가능성이 더 높다.